# 1. File Loading

In [1]:
import torch
import pandas as pd
import numpy as np
import os

# load data
from torch.utils.data import DataLoader
import multiprocessing as mp
import time

# train
from torch import nn
from torch.nn import functional as F
import torch.nn.init

# visualization
import matplotlib.pyplot as plt
from tensorboardX import SummaryWriter

In [2]:
batch_size = 64
num_workers = 40
writer = SummaryWriter('runs/MFCC')
train_set = 'slim_train_set3.csv'
test_set = 'slim_test_set3.csv'

In [3]:
def read_csv(file_name):
    df = pd.read_csv(file_name, header=None)
    return df

file_list = []
for i in range(20):
    file_list.append('./separated data2/train'+str(i+1)+'.csv')

In [4]:
class MFCCDataset(torch.utils.data.Dataset):
    def __init__(self, file_list):
        pool = mp.Pool(processes = 100)
        start = time.time()
        self.raw = pd.concat(pool.map(read_csv, file_list))
        print("time :", time.time() - start)
        pool.close()
        pool.join()
        
        self.label = torch.IntTensor(np.array(self.raw[0].values).reshape(len(self.raw[0].values), 1))
        self.len = len(self.label)
        self.data = torch.Tensor(np.array(self.raw.loc[:,1:]).reshape(len(self.label),100,1000))

    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        return self.data[idx], self.label[idx]

print("Loading data...")

dataset = MFCCDataset(file_list)

dataloader = torch.utils.data.DataLoader(
   dataset, batch_size = batch_size, num_workers = num_workers, drop_last=True
)
print("data is ready!")

Loading data...
time : 28.0802743434906
data is ready!


# 2. Neural Network

In [5]:
import random
USE_CUDA = torch.cuda.is_available() # GPU를 사용가능하면 True, 아니라면 False를 리턴
device = torch.device("cuda" if USE_CUDA else "cpu") # GPU 사용 가능하면 사용하고 아니면 CPU 사용\
print("다음 기기로 학습합니다:", device)
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

다음 기기로 학습합니다: cuda


In [6]:
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # 첫번째층
        # ImgIn shape=(?, 28, 28, 1) -> 1, 100, 1000, 1
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=5, stride=5))

        # 두번째층
        # ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=5, stride=5))

        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer4 = torch.nn.Sequential(
            torch.nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # 전결합층 7x7x64 inputs -> 10 outputs
        self.fc1 = torch.nn.Linear(1 * 10 * 256, 64, bias=True)
        self.fc2 = torch.nn.Linear(64, 3, bias=False)

        # 전결합층 한정으로 가중치 초기화
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)

        dropout = torch.nn.Dropout(p=0.3)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = out.view(out.size(0), -1)   # 전결합층을 위해서 Flatten
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [7]:
# CNN 모델 정의
learning_rate = 0.001
training_epochs = 30
model = CNN()   #.to(device)
model = torch.nn.DataParallel(model)
model.cuda()
criterion = torch.nn.CrossEntropyLoss() #.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
total_batch = len(dataloader)
print('총 배치의 수 : {}'.format(total_batch))

총 배치의 수 : 30


In [8]:
global_step = 0
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in dataloader: # 미니 배치 단위로 꺼내온다. X는 미니 배치, Y는 레이블.
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.reshape(64,1,100,1000).to(device)
        Y = Y.reshape(64,1)[:,0].to(device, dtype=torch.int64)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

        writer.add_scalar('training loss', cost, global_step)
        writer.add_scalar('training avg loss', avg_cost, global_step)
        global_step += 1
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 0.762416422
[Epoch:    2] cost = 0.561019659
[Epoch:    3] cost = 0.336951137
[Epoch:    4] cost = 0.282752723
[Epoch:    5] cost = 0.258081913
[Epoch:    6] cost = 0.241723239
[Epoch:    7] cost = 0.217723101
[Epoch:    8] cost = 0.210497543
[Epoch:    9] cost = 0.189738452
[Epoch:   10] cost = 0.187504828
[Epoch:   11] cost = 0.159189671
[Epoch:   12] cost = 0.174033985
[Epoch:   13] cost = 0.210111946
[Epoch:   14] cost = 0.148172796
[Epoch:   15] cost = 0.147269949
[Epoch:   16] cost = 0.173889533
[Epoch:   17] cost = 0.129776016
[Epoch:   18] cost = 0.0856057853
[Epoch:   19] cost = 0.0733109936
[Epoch:   20] cost = 0.0813689008
[Epoch:   21] cost = 0.0866211951
[Epoch:   22] cost = 0.115619063
[Epoch:   23] cost = 0.0871782526
[Epoch:   24] cost = 0.070455946
[Epoch:   25] cost = 0.0640729144
[Epoch:   26] cost = 0.103766434
[Epoch:   27] cost = 0.129007652
[Epoch:   28] cost = 0.0539382175
[Epoch:   29] cost = 0.0140052857
[Epoch:   30] cost = 0.00550290942


In [9]:
print("Loading testset...")

file_list = []
for i in range(5):
    file_list.append('./separated data2/test'+str(i+1)+'.csv')

testset = MFCCDataset(file_list)

dataloader = torch.utils.data.DataLoader(
   testset, batch_size = batch_size, num_workers = num_workers, shuffle=True, drop_last=True
)
print("testset is ready!")

Loading testset...
time : 23.310343265533447
testset is ready!


In [10]:
with torch.no_grad():
    X_test = testset.data.view(len(testset), 1, 100, 1000).float().to(device)
    Y_test = testset.label.to(device, dtype=torch.int64)[:,0]

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

Accuracy: 0.8948453664779663
